In [1]:
import json
import re
from coreference_resolution import resolve_coreference

In [2]:
# create s_expression_cores examples
data = json.load(open('data/processed_spice_data/dev_each_type_5.json', 'r'))

wikidata_mid_to_fn = json.load(open('data/wikidata_mid_to_fn.json', 'r'))
def sub_mid_to_fn(expression):
    seg_list = expression.split()
    for i in range(len(seg_list)):
        token = seg_list[i].strip(')(')
        if token.startswith('P') or token.startswith('Q'):
            fn = wikidata_mid_to_fn.get(token, "unknown_entity")
            seg_list[i] = seg_list[i].replace(token, fn)
    new_expression = ' '.join(seg_list)
    return new_expression

def process_data(qa):
    return {'coreference_resolved_question': resolve_coreference(qa['question']),
            's_expression_cores_fn': [sub_mid_to_fn(core) for core in qa['s_expression_cores']]}

s_expression_cores_examples = [process_data(data[i]) for i in range(0, len(data), 5)]

json.dump(s_expression_cores_examples, open('data/example/s_expression_cores.json', 'w'), indent=2)

In [ ]:
from utils.parse_expr import get_template_and_replacements, get_simple_question_type

data = json.load(open('data/processed_spice_data/train_full.json', 'r'))

simple_question_types_and_templates = []
for qa in data:
    qa['template'], qa['replacements'] = get_template_and_replacements(qa['s_expression'], qa['s_expression_cores'])
    qa['simple_question_type'] = get_simple_question_type(qa['template'])
    if (qa['simple_question_type'], qa['template']) not in simple_question_types_and_templates:
        simple_question_types_and_templates.append((qa['simple_question_type'], qa['template']))
simple_question_types_and_templates.sort()

for simple_question_type, template in simple_question_types_and_templates:
    print(simple_question_type, "--", template)

compare -- (compare (GROUP_COUNT x1) number)
compare -- (compare (GROUP_COUNT x1) x2)
compare -- (compare (GROUP_SUM (GROUP_COUNT x1) (GROUP_COUNT x2)) (OR x3 x4))
compare -- (compare (GROUP_SUM (GROUP_COUNT x1) (GROUP_COUNT x2)) number)
compare_and_count -- (COUNT (compare (GROUP_COUNT x1) number))
compare_and_count -- (COUNT (compare (GROUP_COUNT x1) x2))
compare_and_count -- (COUNT (compare (GROUP_SUM (GROUP_COUNT x1) (GROUP_COUNT x2)) (OR x3 x4)))
compare_and_count -- (COUNT (compare (GROUP_SUM (GROUP_COUNT x1) (GROUP_COUNT x2)) number))
count -- (COUNT (DISTINCT (OR x1 x2)))
count -- (COUNT (DISTINCT x1))
count -- (COUNT (GROUP_COUNT x1))
count -- (COUNT (GROUP_SUM (GROUP_COUNT x1) (GROUP_COUNT x2)))
count -- (COUNT x1)
optimize -- (optimize (GROUP_COUNT x1))
optimize -- (optimize (GROUP_SUM (GROUP_COUNT x1) (GROUP_COUNT x2)))
simple -- (DIFF x1 x2)
simple -- (DISTINCT x1)
simple -- (GROUP_COUNT x1)
simple -- (GROUP_SUM (GROUP_COUNT x1) (GROUP_COUNT x2))
simple -- (OR x1 x2)
simpl

In [3]:
import random
random.seed(1008)
random.shuffle(data)

In [4]:
from collections import defaultdict
simple_question_type_examples = defaultdict(list)

num = 0
for qa in data:
    simple_question_type = qa['simple_question_type']
    if len(simple_question_type_examples[simple_question_type]) < 3:
        simple_question_type_examples[simple_question_type].append({'coreference_resolved_question': resolve_coreference(qa['question']), 'simple_question_type': qa['simple_question_type']})
        num += 1
        if num == 18:
            break

simple_question_type_examples = [qa for simple_question_type in simple_question_type_examples for qa in simple_question_type_examples[simple_question_type]]
len(simple_question_type_examples)

18

In [ ]:
from collections import defaultdict
import json

wikidata_mid_to_fn = json.load(open('data/wikidata_mid_to_fn.json', 'r'))

def sub_mid_to_fn(expression):
    seg_list = expression.split()
    for i in range(len(seg_list)):
        token = seg_list[i].strip(')(')
        if token.startswith('P') or token.startswith('Q'):
            fn = wikidata_mid_to_fn.get(token, "unknown_entity")
            seg_list[i] = seg_list[i].replace(token, fn)
    new_expression = ' '.join(seg_list)
    return new_expression


template_and_replacements_examples = defaultdict(lambda: defaultdict(list))

each_type_num = 4
num = 0
for qa in data:
    template = qa['template']
    simple_question_type = qa['simple_question_type']
    if template.startswith('(ALL') and template.count('x') > 3:
        continue
    if len(template_and_replacements_examples[simple_question_type][template]) < each_type_num:
        template_and_replacements_examples[simple_question_type][template].append({
                'coreference_resolved_question': resolve_coreference(qa['question']),
                'template': qa['template'],
                'replacements': qa['replacements'],
                's_expression_cores_fn': [sub_mid_to_fn(core) for core in qa['s_expression_cores']],
                's_expression_fn': sub_mid_to_fn(qa['s_expression'])
            })
        num += 1
        if num == 25 * each_type_num:
            break

for simple_question_type in template_and_replacements_examples:
    template_and_replacements_examples[simple_question_type] = [qa for template in template_and_replacements_examples[simple_question_type] for qa in template_and_replacements_examples[simple_question_type][template]]
len(template_and_replacements_examples) # 100

defaultdict(<function __main__.<lambda>()>,
            {'verify': [{'coreference_resolved_question': 'Output: Do Didrik Ficks gränd have Västerlånggatan and Stora Nygatan as terminus?',
               'template': '(ALL x1 x2)',
               'replacements': {'variables': {'x2': 1, 'x1': 0},
                'constants': {},
                'functions': {}},
               's_expression_cores_fn': ['(IS_TRUE Didrik_Ficks_gränd terminus Västerlånggatan)',
                '(IS_TRUE Didrik_Ficks_gränd terminus Stora_Nygatan)'],
               's_expression_fn': '(ALL (IS_TRUE Didrik_Ficks_gränd terminus Västerlånggatan) (IS_TRUE Didrik_Ficks_gränd terminus Stora_Nygatan))'},
              {'coreference_resolved_question': 'Are Canada next to the border of Prince Edward Island and Greenland?',
               'template': '(ALL x1 x2)',
               'replacements': {'variables': {'x2': 1, 'x1': 0},
                'constants': {},
                'functions': {}},
               's_express

In [6]:
json.dump(simple_question_type_examples, open('data/example/simple_question_type.json', 'w'), indent=2)
json.dump(template_and_replacements_examples, open('data/example/template_and_replacements.json', 'w'), indent=2)